# Aula 2: SQL

## Introdução

Nesta aula veremos as principais queries de consulta em sql

## Conexao no db4free via sqlalchemy

In [16]:
!pip install SQLAlchemy --user --upgrade

In [17]:
import yaml 
import sqlalchemy
import pandas as pd

In [6]:
!pip install -U PyYAML

In [7]:
pip install mysqlclient


     |████████████████████████████████| 88 kB 5.0 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.0.3-cp37-cp37m-linux_x86_64.whl size=100142 sha256=e398d63a70b0ea85fb09f7893cbdf12a2d374775fbab39f27ee5929950e7dec6
  Stored in directory: /root/.cache/pip/wheels/79/1c/f8/11fafab45fe6696eea63794a5d747b9c6b54990ac6f1885fb7
Successfully built mysqlclient


In [20]:
with open('credentials.yml', 'r') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

- A engine deve estar no formato:
- mysql://`user`:`password`@`host`:`port`/`database`

In [3]:
print('hello')
print("hello")

hello
hello


In [4]:
sqlalchemy.__version__

'1.4.22'

In [21]:
c = credentials
engine = sqlalchemy.create_engine\
(f"mysql://{c['user']}:{c['password']}@{c['host']}:{c['port']}/{c['database']}")

### UPLOAD TABLE

In [10]:
df = pd.read_csv('AtrasoCartaoCredito.csv')

In [ ]:
df.head(2)

,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737


In [13]:
!pip uninstall mysql-connector


In [14]:
!pip install mysql-connector-python


     |████████████████████████████████| 30.9 MB 82 kB/s 


In [22]:
import mysql.connector

#Conectando
con = mysql.connector.connect(user=credentials['user'], 
                              password=credentials['password'],
                              host=credentials['host'],
                              database=credentials['database'],
                              port=credentials['port'])

cursor = con.cursor()

In [23]:
query = 'SHOW TABLES'
cursor.execute(query)
cursor.fetchall()

[('empresas',)]

In [31]:
df.to_sql('AtrasoCartaoCredito', engine)

In [32]:
type(df)

pandas.core.frame.DataFrame

In [33]:
df.head()

,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333


### SELECT e FROM

- Já vimos o `SELECT` e `FROM` na última aula
    - `SELECT` seleciona as colunas que queremos trazer
        - `*` seleciona toda das colunas
    - `FROM` seleciona a tabela que analisaremos
- Para facilitar a consulta, podemos aplicar o `LIMIT`, que define o número de linhas que a query irá trazer

In [34]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
2,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
3,3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
4,4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,887,0,2,homem,27,0,13.0,0.928571,1.083333,BA,Marcionílio Souza,Região Nordeste,0.080247
887,887,888,1,1,mulher,19,0,30.0,2.142857,2.500000,BA,Mirante,Região Nordeste,0.263158
888,888,889,0,3,mulher,35,1,2345.0,167.500000,195.416667,RO,Governador Jorge Teixeira,Região Norte,11.166667
889,889,890,1,1,homem,26,0,30.0,2.142857,2.500000,PI,Baixa Grande do Ribeiro,Região Nordeste,0.192308


### LIMIT

- Podemos usar o `LIMIT` para mostrar as primeiras `n` linhas

In [35]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
2,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
3,3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
4,4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333


- Atenção, em bancos de dados SQLServer, o análogo ao `LIMIT` é o `TOP`
- A sintaxe análoga seria:
    - `SELECT TOP 5 * FROM AtrasoCartaoCredito`

### WHERE

- Podemos filtar algumas observações com o comando `WHERE`

#### Igual `=`

In [38]:
df.shape

(891, 13)

In [39]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE Sexo = 'homem'
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333
2,5,6,0,3,homem,22,0,84583.0,6041.642857,7048.583333,AL,Carneiros,Região Nordeste,640.780303
3,6,7,0,1,homem,54,0,51862.0,3704.428571,4321.833333,PR,Figueira,Região Sul,160.067901
4,7,8,0,3,homem,20,3,21075.0,1505.357143,1756.250000,BA,Ouriçangas,Região Nordeste,175625.000000


#### Diferente `<>`

- `<>` significa "diferente", ou seja, selecionaremos todos `sexo` menos `homem`

In [40]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE Sexo <> 'homem'
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
1,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
2,3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
3,8,9,1,3,mulher,27,0,11133.0,795.214286,927.750000,MG,Comercinho,Região Sudeste,68.722222
4,9,10,1,2,mulher,14,1,3007.0,214.785714,250.583333,MG,Japonvar,Região Sudeste,35.797619


#### Maior `>`

In [41]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE GastoMedio > 1000
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
1,5,6,0,3,homem,22,0,84583.0,6041.642857,7048.583333,AL,Carneiros,Região Nordeste,640.780303
2,6,7,0,1,homem,54,0,51862.0,3704.428571,4321.833333,PR,Figueira,Região Sul,160.067901
3,7,8,0,3,homem,20,3,21075.0,1505.357143,1756.250000,BA,Ouriçangas,Região Nordeste,175625.000000
4,13,14,0,3,homem,39,1,31275.0,2233.928571,2606.250000,BA,Abaíra,Região Nordeste,133.653846


In [43]:
df[df.GastoMedio > 1000].head(5)

,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
5,6,0,3,homem,22,0,84583.0,6041.642857,7048.583333,AL,Carneiros,Região Nordeste,640.780303
6,7,0,1,homem,54,0,51862.0,3704.428571,4321.833333,PR,Figueira,Região Sul,160.067901
7,8,0,3,homem,20,3,21075.0,1505.357143,1756.250000,BA,Ouriçangas,Região Nordeste,175625.000000
13,14,0,3,homem,39,1,31275.0,2233.928571,2606.250000,BA,Abaíra,Região Nordeste,133.653846


#### Entre `BETWEEN`

In [44]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE GastoMedio BETWEEN 500 AND 1000
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
2,8,9,1,3,mulher,27,0,11133.0,795.214286,927.750000,MG,Comercinho,Região Sudeste,68.722222
3,19,20,1,3,mulher,45,0,7225.0,516.071429,602.083333,SE,Ilha Das Flores,Região Nordeste,26.759259
4,26,27,0,3,homem,31,0,7225.0,516.071429,602.083333,PB,Gado Bravo,Região Nordeste,38.844086


#### E `AND`

In [45]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE GastoMedio > 1000 AND Sexo = 'mulher'
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
1,14,15,0,3,mulher,14,0,78542.0,5610.142857,6545.166667,MG,Alto Jequitibá,Região Sudeste,935.023810
2,22,23,1,3,mulher,15,0,80292.0,5735.142857,6691.000000,GO,Aragoiânia,Região Centro-Oeste,892.133333
3,24,25,0,3,mulher,80,3,21075.0,1505.357143,1756.250000,TO,Alvorada,Região Norte,439.062500
4,25,26,1,3,mulher,30,1,31387.0,2241.928571,2615.583333,RR,Uiramutã,Região Norte,174.372222


#### Ou `OR`

In [46]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE GastoMedio > 1000 OR CidadeResidencia = 'Agudos do Sul'
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
1,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
2,5,6,0,3,homem,22,0,84583.0,6041.642857,7048.583333,AL,Carneiros,Região Nordeste,640.780303
3,6,7,0,1,homem,54,0,51862.0,3704.428571,4321.833333,PR,Figueira,Região Sul,160.067901
4,7,8,0,3,homem,20,3,21075.0,1505.357143,1756.250000,BA,Ouriçangas,Região Nordeste,175625.000000


#### Em `IN`

In [47]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE UF = 'BA' OR UF = 'PR'
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
1,3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
2,4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333
3,6,7,0,1,homem,54,0,51862.0,3704.428571,4321.833333,PR,Figueira,Região Sul,160.067901
4,7,8,0,3,homem,20,3,21075.0,1505.357143,1756.250000,BA,Ouriçangas,Região Nordeste,175625.000000


In [48]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE UF IN ('BA', 'PR')
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
1,3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
2,4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333
3,6,7,0,1,homem,54,0,51862.0,3704.428571,4321.833333,PR,Figueira,Região Sul,160.067901
4,7,8,0,3,homem,20,3,21075.0,1505.357143,1756.250000,BA,Ouriçangas,Região Nordeste,175625.000000


#### Não `NOT`

In [49]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE UF NOT IN ('BA', 'PR')
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
2,5,6,0,3,homem,22,0,84583.0,6041.642857,7048.583333,AL,Carneiros,Região Nordeste,640.780303
3,8,9,1,3,mulher,27,0,11133.0,795.214286,927.750000,MG,Comercinho,Região Sudeste,68.722222
4,9,10,1,2,mulher,14,1,3007.0,214.785714,250.583333,MG,Japonvar,Região Sudeste,35.797619


#### `LIKE`

In [50]:
query = '''
    SELECT *
    FROM AtrasoCartaoCredito
    WHERE UF LIKE 'A_'
    LIMIT 5
'''

pd.read_sql(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,5,6,0,3,homem,22,0,84583.0,6041.642857,7048.583333,AL,Carneiros,Região Nordeste,640.780303
1,37,38,0,3,homem,21,0,805.0,57.500000,67.083333,AL,Porto de Pedras,Região Nordeste,6.388889
2,41,42,0,2,mulher,27,1,21.0,1.500000,1.750000,AM,Silves,Região Norte,0.129630
3,46,47,0,3,homem,34,1,155.0,11.071429,12.916667,AC,Bujari,Região Norte,0.759804
4,47,48,1,3,mulher,15,0,775.0,55.357143,64.583333,AL,Olho D´água do Casado,Região Nordeste,8.611111


### DISTINCT

- Podemos analisar os valores únicos de uma coluna

In [51]:
query = '''
    SELECT DISTINCT GrupoEconomico
    FROM AtrasoCartaoCredito
'''

pd.read_sql(query, engine)

,GrupoEconomico
0,3
1,1
2,2


In [52]:
query = '''
    SELECT DISTINCT UF
    FROM AtrasoCartaoCredito
'''

pd.read_sql(query, engine)

,UF
0,MG
1,RN
2,PR
3,BA
4,AL
5,RO
6,PB
7,SP
8,RS
9,SC
